# 机器学习实现之模型评估二

目录

逻辑回归模型
皮马印第安人糖尿病数据集
使用精确率进行模型评估的局限性
混淆矩阵、模型衡量指标及其意义

##皮马印第安人糖尿病数据集

基于数据集中包括的某些诊断测量来诊断性的预测患者是否患有糖尿病
输入的特征有：患者的怀孕次数，葡萄糖量，血压，皮褶厚度，胰岛素水平，糖尿病谱系功能，年龄等
输出结果：是否患有糖尿病
数据来源：https://www.kaggle.com/uciml/pima-indians-diabetes-database

任务：选取怀孕次数，胰岛素水平，体重指数，年龄这四个特征对是否患有糖尿病进行预测

In [14]:
# 数据预处理

import pandas as pd
path = 'dataset/pima_data.csv'
pima = pd.read_csv(path)
pima.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [19]:
# X, y的赋值
feature_names = ['Pregnancies','Insulin','BMI','Age']
X = pima[feature_names]
y = pima.Outcome

In [20]:
# 维度的确认
print(X.shape)
print(y.shape)

(768, 4)
(768,)


In [25]:
#数据分离
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)#test_size(test/train)默认为0.25

In [22]:
# 模型训练
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train , y_train)

G:\anaconda3\envs\sklearn\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [31]:
# 基于测试数据集的结果预测
y_pred = logreg.predict(X_test)

In [32]:
# 使用准确率进行评估
from sklearn import metrics
print(metrics.accuracy_score(y_test,y_pred))

0.6927083333333334


预测准确率的局限性：
无法真实反映模型针对各个分类的预测准确度
准确率可以方便的用于衡量模型的整体预测效果，但无法反应细节信息，具体表现：
没有体现数据的实际分布情况
没有体现模型错误预测的类型
空准确率：当模型总是预测比例较高的类别，其预测准确率的数值

In [34]:
# 确认正负样本数量
y_test.value_counts()

0    130
1     62
Name: Outcome, dtype: int64

所以可见其中负样本的比例较高，高达0.67  由此可见即使全部直接预测为负样本即可达到0.67的准确率，所以前面的百分之69的准确率局限性较大

In [40]:
# 计算并展示混淆矩阵
confusion = metrics.confusion_matrix(y_test,y_pred)
print(confusion)

#四个因子赋值
TN = confusion[0,0]
FP = confusion[0,1]
FN = confusion[1,0]
TP = confusion[1,1]
print(TN,FP,FN,TP)

[[118  12]
 [ 47  15]]
118 12 47 15


In [37]:
# 部分预测结果与实际情况对比展示
print("true:",y_test.values[0:25])
print("pred:",y_pred[0:25])

true: [1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0]
pred: [0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]


# 混淆矩阵指标计算

准确率：整体样本中，预测正确样本数的比例
Accuracy = （TP + TN）/(TP + TN + FP + FN)

In [43]:
accuracy = (TP + TN)/(TP + TN + FP + FN)
print(accuracy)

0.6927083333333334


错误率：整体样本中，预测错误的比例
Misclassification Rate = （FP + FN）/(TP + TN + FP + FN)

In [44]:
mis_rate = (FP + FN)/(TP + TN + FP + FN)
print(mis_rate)

0.3072916666666667


召回率：正样本中，预测正确的比例
recall = TP/(TP + FN)

In [45]:
recall = TP/(TP + FN)
print(recall)

0.24193548387096775


特异度：负样本中，预测正确的比例
Specificity = TN/(TN + FP)

In [46]:
specificity = TN/(TN + FP)
print(specificity)

0.9076923076923077


精确率：预测结果为正样本中，预测正确的比例
Precision = TP/(TP + FP)

In [47]:
precision = TP/(TP + FP)
print(precision)

0.5555555555555556


F1分数：综合Precision和Recall的判断指标

In [48]:
f1_score = 2 * precision * recall/(precision + recall)
print(f1_score)

0.3370786516853933


结论：

分类任务中，相比单一的预测准确率，混淆矩阵提供了更全面的模型评估信息
通过混淆矩阵，我们可以计算出多样性的模型表现衡量指标，从而更好地选择模型


哪个衡量指标更关键？

衡量指标的选择取决于应用场景
垃圾邮件检测（正样本判断为“垃圾邮件”）：希望普通邮件（负样本）不要被判断为垃圾邮件（正样本），需要关注精确率和召回率
异常交易检测（正样本为“异常交易”）：希望所有的异常交易都被检测到，需要关注特异度